In [1]:
%load_ext autoreload
%autoreload

In [2]:
import gzip
from Bio import SeqIO



In [3]:
%%time

gbrec = list(SeqIO.parse('../data/GCF_000005845.2_ASM584v2_genomic.gbff', 'genbank'))[0]

CPU times: user 1.14 s, sys: 79.5 ms, total: 1.21 s
Wall time: 1.27 s


In [4]:
gene_features = [f for f in gbrec.features if f.type == 'gene']

In [5]:
%%time

len(gbrec.seq)

CPU times: user 13 µs, sys: 1 µs, total: 14 µs
Wall time: 19.1 µs


4641652

In [6]:
def tileset_info(gbrec):
    ret_val = {
          'tile_size': 1024,
          'max_zoom': 22,
          'max_width': len(gbrec.seq),
          'min_pos': [0],
          'max_pos': [len(gbrec.seq)]
    }
    return ret_val;

tsinfo = tileset_info(gbrec)
print(tsinfo)

{'tile_size': 1024, 'max_zoom': 22, 'max_width': 4641652, 'min_pos': [0], 'max_pos': [4641652]}


In [66]:
import slugid

def filler_to_genepred(filler, strand):
    return {
        'xStart': filler['start'],
        'xEnd': filler['end'],
        'type': 'filler',
        'uid': slugid.nice(),
        'fields': [],
        'strand': strand,
    }

def gb_to_genepred(gb):
    importance = gb.location.end - gb.location.start;
    strand = '+' if gb.strand == 1 else '-'
    uid = slugid.nice();
    
    if gb.type == 'filler':
      # this is annotation that was generated by collapsing genes and is
      # only meant to show that there is something there.
        return {
            'xStart': gb.location.start,
            'xEnd': gb.location.end,
            'strand': gb.strand,
            'fields': [],
            'type': 'filler',
            'uid': uid,
        }
      
    name = gb.qualifiers['gene'][0]
    start = int(gb.location.start)
    end = int(gb.location.end)
    
    return {
      'xStart': int(gb.location.start),
      'xEnd': int(gb.location.end),
      'strand': strand,
      'chrOffset': 0,
      'importance': gb.location.end - gb.location.start,
      'uid': uid,
      'fields': [
        'chrom', start, end,
          name, importance, strand, '', '', gb.type,
          name, str(start), str(end),
          str(start), str(end)
      ]
    }

gb_to_genepred(gene_features[0])

{'xStart': 189,
 'xEnd': 255,
 'strand': '+',
 'chrOffset': 0,
 'importance': 66,
 'uid': 'G7HKm0gqSZyTk5ucfReqmw',
 'fields': ['chrom',
  189,
  255,
  'thrL',
  66,
  '+',
  '',
  'gene',
  'thrL',
  '189',
  '255',
  '189',
  '255']}

In [67]:
"""
* Take a list of genes, which can be any list with elements containing
* { start, end } fields and return another list of { start, end } 
* fields containing the collapsed genes. 
*
* The segments should be sorted by their start coordinate.
*
* The scale parameter is the number of base pairs per pixels
"""
def collapse(segments, scale):
    collapsed = []

    # the maximum distance we allow between segments before collapsing them
    MAX_DIST_BETWEEN = 5

    # no segments in, no segments out
    if not len(segments):
      return []

    #start with the first segment
    curr_start = segments[0].location.start;
    curr_end = segments[0].location.end;

    # continue on to the next segments
    for segment in segments:
        if segment.location.start < curr_end + MAX_DIST_BETWEEN * 1 / scale:
            curr_end = max(curr_end, segment.location.end)
        else:
            collapsed += [{
                'type': 'filler',
                'start': curr_start,
                'end': curr_end,
            }]
            
            curr_start = segment.location.start
            curr_end = segment.location.end

    # add the final segment
    collapsed += [{
        'start': curr_start,
        'end': curr_end,
        'type': 'filler',
    }]

#     print('collapsed:', collapsed)
    return collapsed

In [68]:
tiles(gbrec, tsinfo, 0, 0)

[{'xStart': 189,
  'xEnd': 255,
  'strand': '+',
  'chrOffset': 0,
  'importance': 66,
  'uid': 'XGVsdT-4SnS5dqPOnP1U8g',
  'fields': ['chrom',
   189,
   255,
   'thrL',
   66,
   '+',
   '',
   'gene',
   'thrL',
   '189',
   '255',
   '189',
   '255']},
 {'xStart': 336,
  'xEnd': 2799,
  'strand': '+',
  'chrOffset': 0,
  'importance': 2463,
  'uid': 'RBUyFTetQ66ZOUs1ab3IHw',
  'fields': ['chrom',
   336,
   2799,
   'thrA',
   2463,
   '+',
   '',
   'gene',
   'thrA',
   '336',
   '2799',
   '336',
   '2799']},
 {'xStart': 2800,
  'xEnd': 3733,
  'strand': '+',
  'chrOffset': 0,
  'importance': 933,
  'uid': 'cLMNdv4zRCy8jNhfUqbMSQ',
  'fields': ['chrom',
   2800,
   3733,
   'thrB',
   933,
   '+',
   '',
   'gene',
   'thrB',
   '2800',
   '3733',
   '2800',
   '3733']},
 {'xStart': 3733,
  'xEnd': 5020,
  'strand': '+',
  'chrOffset': 0,
  'importance': 1287,
  'uid': 'dSEV5R_NRBS1q8CN-sPW_w',
  'fields': ['chrom',
   3733,
   5020,
   'thrC',
   1287,
   '+',
   '',
   'gene',

In [69]:
def tiles(gbrec, tsinfo, z, x):
    tile_width = tsinfo['max_width'] / 2 ** z;
    

    # get the bounds of the tile
    min_x = tsinfo['min_pos'][0] + x * tile_width;
    max_x = tsinfo['min_pos'][0] + (x + 1) * tile_width;

    gene_features = [f for f in gbrec.features if f.type == 'gene']
    
    filtered = list(filter(
        lambda x: x.type == 'gene' and x.location.end > min_x and x.location.start < max_x,
        gene_features))
    scale_factor = 1024 / (2 ** (tsinfo['max_zoom'] - z));
    
    collapsed_plus = [{**c, **{'strand': '+'}} for c in
        collapse(list(filter(lambda x: x.strand == 1, filtered)), scale_factor)]
    collapsed_minus = [{**c, **{'strand': '-'}} for c in
        collapse(list(filter(lambda x: x.strand != 1, filtered)), scale_factor)]

    values = []
    tile_capacity = 20
    
    for feature in gene_features:
        if len(values) >= tile_capacity:
            break
        
        if feature.location.end >= min_x and feature.location.start <= max_x:
            values += [gb_to_genepred(feature)]
    
    values = (values + 
              [filler_to_genepred(c, '+') for c in collapsed_plus] + 
              [filler_to_genepred(c, '-') for c in collapsed_minus])

    return values # .map(x => gbToHgGene(x));


In [70]:
tiles(gbrec, tsinfo, 0, 0)

[{'xStart': 189,
  'xEnd': 255,
  'strand': '+',
  'chrOffset': 0,
  'importance': 66,
  'uid': 'Tb_w2j9TQXiA7ZBbIALqUQ',
  'fields': ['chrom',
   189,
   255,
   'thrL',
   66,
   '+',
   '',
   'gene',
   'thrL',
   '189',
   '255',
   '189',
   '255']},
 {'xStart': 336,
  'xEnd': 2799,
  'strand': '+',
  'chrOffset': 0,
  'importance': 2463,
  'uid': 'YKMUot85SQeowv90JTfflg',
  'fields': ['chrom',
   336,
   2799,
   'thrA',
   2463,
   '+',
   '',
   'gene',
   'thrA',
   '336',
   '2799',
   '336',
   '2799']},
 {'xStart': 2800,
  'xEnd': 3733,
  'strand': '+',
  'chrOffset': 0,
  'importance': 933,
  'uid': 'DLE11LAdT0Kvb0p6Nmswkw',
  'fields': ['chrom',
   2800,
   3733,
   'thrB',
   933,
   '+',
   '',
   'gene',
   'thrB',
   '2800',
   '3733',
   '2800',
   '3733']},
 {'xStart': 3733,
  'xEnd': 5020,
  'strand': '+',
  'chrOffset': 0,
  'importance': 1287,
  'uid': 'fmsiWIhURLqFytHd4qAKqg',
  'fields': ['chrom',
   3733,
   5020,
   'thrC',
   1287,
   '+',
   '',
   'gene',

In [71]:
def tiles_wrapper(tile_ids, tile_func):
    tile_values = []

    for tile_id in tile_ids:
        parts = tile_id.split('.')

        if len(parts) < 3:
            raise IndexError("Not enough tile info present")

        z = int(parts[1])
        x = int(parts[2])

        ret_array = tile_func(z, x)

        tile_values += [(tile_id, ret_array)]

    return tile_values

In [72]:
#tiles_wrapper(['x.0.0'], lambda z,x: tiles(gbrec, tsinfo, z, x))

In [83]:
from higlass.tilesets import Tileset

def gbk(gbrec):
    """1d labels"""
    return Tileset(
        uuid='hi',
        tileset_info=lambda: tileset_info(gbrec),
        tiles=lambda tids: tiles_wrapper(tids, lambda z,x: tiles(gbrec, tsinfo, z, x))
    )

In [84]:
ts = gbk(gbrec)

In [85]:
import higlass
from higlass.client import View, Track

(display, server, viewconf) = higlass.display([
    View([
        Track('top-axis'),
        Track(
            track_type='horizontal-gene-annotations',
            position='top',
            tileset=ts,
            height=100)
    ])
], server_port=8112)

display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'b-BE01_xSBON1YitSsaj3A', 'tracks': {'top': [{'ty…

In [76]:
server

In [77]:
server.port

64406